# Машинное обучение

## Факультет математики НИУ ВШЭ

### 2018-2019 учебный год

Лектор: Илья Щуров

Семинаристы: Евгения Ческидова, Евгений Ковалев

Ассистенты: Константин Ваниев, Софья Дымченко

# Семинар 3

Сегодня мы узнаем

* Что такое обучение с учителем, что такое непараметрические и параметрические алгоритмы ml
* Как работает метод ближайших соседей, какие у него есть параметры и как они влияют
* Что такое скользящий контроль и какой он бывает
* Узнаем о разложении ошибки на смещение и разброс
* Узнаем о влиянии размерности пространства признаков на эффективность алгоритма knn

# Метод K ближайших соседей
## K nearest neighbours

Алгоритм KNN является надежным и универсальным классификатором, который часто используется в качестве бейзлайна для более сложных классификаторов, таких как искусственные нейронные сети (ANN). Несмотря на свою простоту, KNN может превзойти более мощные классификаторы и используется во множестве приложений, таких как экономическое прогнозирование, сжатие данных и генетика. Например, KNN использовалась в [исследовании](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-7-S1-S11) по функциональной геномике в 2006 году, где гены определялись на основе их профилей экспрессии.

# Что такое алгоритм KNN?

Начнем с введения некоторых определений и обозначений.

* Мы будем использовать $x$ для обозначения вектора признаков (или атрибутов) объекта.
* Под $y$ мы будем подразумевать метку или класс, который мы пытаемся предсказать.

**Обучение с учителем**

KNN входит в число **supervised** алгоритмов или алгоритмов "обучения с учителем".
Это означает, что нам предоставляется размеченный набор данных, для которого известны соответствия между наблюдениями $(x, y)$. Целью является на основе предоставляемой выборки найти связь между $x$ и $y$, чтобы восстановить функцию $h: X \rightarrow Y$. Имея такую функцию мы можем предсказать $y$ по имеющемуся наблюдению $x$.

**Классификатор KNN также является непараметрическим алгоритмом обучения**

Непараметрический означает, что алгоритм не делает явных предположений о функциональной форме $h$, избегая опасностей ошибочного определения лежащего в основе распределения данных. Например, предположим, что наши данные сильно негауссовы, но выбранная нами модель обучения делает сильное предположение о гауссовой форме распределения. В этом случае такой алгоритм сделает крайне плохие прогнозы.

Обучение KNN заключается в запоминании экземпляров обучающей выборки, которые впоследствии используются как «знание» для фазы прогнозирования. Конкретно это означает, что только когда запрос в нашу базу данных сделан (т.е. когда мы просим KNN предсказать метку с учетом ввода), алгоритм будет использовать экземпляры обучения, чтобы выдать ответ.

**С какими сложностями можно столкнуться при обучении алгоритма KNN?**

Нужно понимать, что фаза обучения, заключающаяся в запоминании объектов выборки не является дорогостоящей по времени.
Однако этого нельзя сказать о фазе тестирования. Если база обучения достаточно большая, осуществление поиска по ней достаточно дорогостоящий процесс.


# Как KNN работает?

<img src="https://kevinzakka.github.io/assets/1nearestneigh.png" width="400" align="center">
<img src="https://kevinzakka.github.io/assets/20nearestneigh.png" width="400" align="center">

В случае классификации алгоритм K-ближайших соседей по существу сводится к подсчету большинства голосов между K наиболее похожими экземплярами для данного тестового экземпляра. Сходство экземпляров определяется по расстоянию между двумя точками данных. Популярным выбором является евклидово расстояние, но другие меры близости также могут быть подходящими для конкретной ситуации, включая расстояние Манхэттен, Чебышева и Хэмминга.


# Знакомство с датасетом

Сначала рассмотрим как работает уже готовый алгоритм из sklearn на примере датасета с разными видами стекла.

Подробнее о датасете тут: [kaggle_dataset_link](https://www.kaggle.com/uciml/glass)

Изучим вместе с KNN методы кросс-валидации, метрики и подбор параметров.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("./data/glass.csv")
print("Размерность данных: {}\n".format(data.shape))
data.head()

Сколько есть видов стекла в нашем датасете?

In [ ]:
data.Type.unique()

Посмотрим на распределения признаков по разным классам;

Рассмотрим для начала небольшую подвыборку признаков просто для удобства отрисовки.

In [ ]:
import seaborn as sns
%matplotlib inline

In [ ]:
sns.pairplot(data.iloc[:, [1,2,3,-1]], hue='Type')

Видно, что не находится такой пары признаков, в координатах которых типы стекол легко бы разделялись. Также видно, что датасет неуравновешен и есть большой перекос в количестве примеров в сторону 1 и 2 классов. Однако в случае использования KNN для нас это не является большой проблемой. Почему?

## Кросс-валидация (Скользящий контроль)

**Терминология**
<img src="http://www.vias.org/tmdatanaleng/img/hl_crossval.png" width="400" align="center">

**Оценкой скользящего контроля** называется средняя по всем разбиениям величина ошибки на контрольных подвыборках.

Если выборка независима, то средняя ошибка скользящего контроля даёт несмещённую оценку вероятности ошибки.

Выделяют следующие виды скользящего контроля

* *Контроль на отложенных данных* (**hold-out CV**)
    - Оценка производится по одному случайному разбиению 
* *Контроль по отдельным объектам* (**leave-one-out CV**)
    - Поочередно из выборки убирается один объект, модель обучется на оставшейся выборке и делает предсказание на отложенном объекте.
* *Контроль по $q$ блокам (**q-fold CV**)*
    - Выборка случайным образом разбивается на $q$ непересекающихся блоков одинаковой (или почти одинаковой) длины.

Подробнее про каждый из них и про доверительные интервалы для оценок результатов кросс-валидации можно почитать [здесь](http://www.machinelearning.ru/wiki/index.php?title=Скользящий_контроль).

Еще один важный термин

**Стратификация** заключается в том, чтобы заранее поделить выборку на части (страты), и при разбиении на обучение длины $m$ и контроль длины $k$ гарантировать, что каждая страта будет поделена между обучением и контролем в той же пропорции $m:k$.

**Стратификация классов в задачах классификации** означает, что каждый класс (который и определяет страту) делится между обучением и контролем в пропорции $m:k$, где $m$ — размер обучающей выборки, а $k$ — размер тестовой выборки при данном варианте разбиения.
Стратификация позволяет уменьшить разброс оценок скользящего контроля и сузить доверительные интервалы для них.

# sklearn

В sklearn есть встроенные средства для деления датасета на обучение и тест.

In [ ]:
X, y = data.iloc[:, :-1].values, data.Type.values

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold

# стратификация согласно разметке по классам
# деление датасета на обучение и тест один раз
# это и есть hold-out CV
# как правило при таком подходе k-fold cv производится отдельно над train частью датасета
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=data.Type)

In [ ]:
# реализация q-fold cv со стратификацией по параметру y

kfold_iterator = StratifiedKFold(n_splits=3, random_state=42)

for train_index, test_index in kfold_iterator.split(X,y):
    print("TRAIN:", train_index[:5], "TEST:", test_index[:5])
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

Другие виды кросс-валидации также доступны и с ними можно ознакомиться [здесь](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection)

## Исследуем KNN из коробки

KNN имеет следующие важные параметры, которые напрямую затрагивают результаты предсказания
    * weights
    * n_neighbors
    
Прочитайте в документации про них подробнее.

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# выбираем только нечетные значения k от 1 до 50
neighbors = np.arange(1, 50, 2)

# сюда мы сложим наши скоры
cv_scores = []

kf = StratifiedKFold(n_splits=7, random_state=42, shuffle=True)

# выполним кросс-валидацию на 7 фолдах со стратификацией,
# в качестве метрики возьмем простое accuracy
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k, weights='uniform')
    scores = cross_val_score(knn, X, y, cv=kf, scoring='accuracy',)
    cv_scores.append(scores.mean())
    
# отрисуем полученный результат
plt.scatter(neighbors, cv_scores, label='best k={},\n acc={}'.format(neighbors[np.argmax(cv_scores)],
                                                               np.round(max(cv_scores),2)), )
plt.ylabel("Accuracy")
plt.legend()
plt.xlabel("K")
plt.show()

# Задания
### Проведите эксперименты, чтобы ответить на следующие вопросы ~~и получше прочувствовать knn~~

**Удобно код экспериментов писать в ячейках ниже задания с пометкой c номером задания**

1. Для датасета со стеклом. Воспользуйтесь `StratifiedKFold` с количеством фолдов = 5. На каждом шаге кросс-валидации запоминайте precision & recall по каждому отдельному классу на текущей тестовой части выборки. В конце постройте графики отдельно для precision и отдельно для recall, где каждому классу будет сопоставлена усредненная по 5 фолдам метрика. 

    * Насколько сильно различаются предсказания knn для разных классов?

    * Соответствует ли полученное распределение метрик по классам исходному распределению классов?

    * Как изменится распределение метрик при изменении параметра k?

2. Проведите эксперименты с одинаковым разбиением для cv, но различными значениями параметра `weights` у KNN.
Постройте 2 графика в одних координатах: для значения среднего `f1_macro` на кросс-валидации для `uniform`, `distance` knn в зависимости от k. Сделайте вывод о влиянии параметра weights на предсказание модели для данной задачи.

3. Постройте зависимость разброса предсказаний модели от k. В качестве cv возьмите StratifiedKFold c n_splits=7. Для этого удобно воспользоваться методом `kneighbors` у knn классификатора.
Опишите свои наблюдения, сделайте выводы. 

4. Проведите эксперимент по уменьшению размерности векторов признаков в датасете#2. Для этой цели воспользуйтесь [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). Постройте график зависимости accuracy от размерности входных данных. Эксперимент проводите при фиксированных параметрах KNN. Сделайте вывод.

5.(optional) Воспользуйтесь датасетом, определенным в следующей ячейке (в дальнейшем датасет#2). Разбейте его на трейн и тест с `random_state`=42, `test_size`=0.3, `stratify` по y.
Поэкспериментируйте с различными вариантами cv на трейн части и постарайтесь выбрать наилучшие параметры knn.
После того, как параметры будут подобраны обучите модель на train и сделайте с ее помощью предсказание на test. Какое наилучшее значение accuracy у вас получилось на test части и при каких параметрах knn?

    Задание считается успешно выполненным, если итоговый accuracy > 0.68



# Самостоятельная часть

Для задания **#1**

In [ ]:
from sklearn.metrics import precision_score, recall_score

<your code>

Для задания **#2**

In [ ]:
neighbors = np.arange(1, 50, 2)

<your code>

Вероятно, что `distance` довольно полезная штука и стоит ее пользоваться, также можно придумать свои функции взвешивания и попытаться превзойти такое вот простое взвешивание.

Для задания **#3**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=data.Type)

neighbors = np.arange(1, 100, 2)

# сюда мы сложим наши скоры
variances = []
kf = StratifiedKFold(n_splits=7, random_state=42, shuffle=True)

<your code>

Для задания **#4**

`проклятие размерности`

In [ ]:
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

# датасет#2
X,y = make_classification(n_samples=500, n_features=1000, n_informative=900, n_classes=2)
scores = []
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

<your code>